In [1]:
import pandas as pd
import numpy as np

df_wide = pd.read_csv('./input/ch03/time_series_wide.csv', index_col = 0)
df_wide.index = pd.to_datetime(df_wide.index)

In [2]:
print(df_wide.iloc[:5,:3])

              A     B     C
2016-07-01  532  3314  1136
2016-07-02  798  2461  1188
2016-07-03  823  3522  1711
2016-07-04  937  5451  1977
2016-07-05  881  4729  1975


In [ ]:
df_long = df_wide.stack().reset_index(1)